<a href="https://colab.research.google.com/github/piyengar/vehicle-predictor/blob/master/Vehicle_color_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
! pip install pytorch-lightning
! pip install pytorch-lightning-bolts
! pip install ipywidgets
! pip install torchmetrics

In [2]:
import os

import torch
import numpy as np
import pandas as pd 
import xml.etree.ElementTree as et 
import pytorch_lightning as pl
import matplotlib.pyplot as plt
from typing import Optional, List, Any
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset, random_split, ConcatDataset
from torchvision import transforms
from torchvision.models import resnet18, squeezenet1_1, resnet50, resnet152
from pytorch_lightning.callbacks import (BasePredictionWriter, 
                                         LearningRateMonitor, 
                                         ModelCheckpoint, 
                                         EarlyStopping,
                                         )
# from pl_bolts.datasets import DummyDataset
from PIL import Image
from ipywidgets import interact
from IPython.display import Image as ipImage
from torchmetrics import ConfusionMatrix, Accuracy, Precision, Recall, F1
from torchmetrics.functional import accuracy, confusion_matrix, f1, precision, recall
from sklearn.metrics import ConfusionMatrixDisplay


In [21]:
# Prediction Params -- one of : VRIC, Cars196, VehicleID, BoxCars116k, CompCars, Veri, Combined
train_dataset_name = 'Combined'
# Path constants
STORAGE_ROOT='/content/drive'
CHECKPOINT_ROOT=os.path.join(STORAGE_ROOT, 'MyDrive/Gatech/CARZAM/checkpoints/color', train_dataset_name)
PREDICTION_ROOT=os.path.join(STORAGE_ROOT, 'MyDrive/Gatech/CARZAM/predictions/color', train_dataset_name)
DATASET_ROOT=os.path.join(STORAGE_ROOT, 'MyDrive/Gatech/CARZAM/Datasets')
GITHUB_PAT_PATH=os.path.join(STORAGE_ROOT, 'MyDrive/Gatech/github_pat_colab.txt')

with open(GITHUB_PAT_PATH) as reader:
    GITHUB_PAT = reader.readline()


In [4]:
from google.colab import drive
drive.mount(STORAGE_ROOT)
!mkdir -p "{CHECKPOINT_ROOT}"
!mkdir -p "{PREDICTION_ROOT}"

Mounted at /content/drive


In [23]:
!git clone "https://{GITHUB_PAT}@github.com/piyengar/vehicle-predictor.git" ./code

Cloning into './code'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 31 (delta 12), reused 14 (delta 5), pack-reused 0
Unpacking objects: 100% (31/31), done.
